In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.linkedin.com/jobs/view/4361351616/?alternateChannel=search&eBP=NON_CHARGEABLE_CHANNEL&trk=d_flagship3_search_srp_jobs&refId=zrSLffHGI0vnHEciLfcAMA%3D%3D&trackingId=LFfadPd%2FnjDBlcFTAM7F4Q%3D%3D")
docs = loader.load()

In [20]:
docs[0].page_content

"\n\n\n\n\n\n \n \n\n\n\n\n\n\n\n\n\nLinkedIn hiring AI/ML Engineer Intern, Master's - Summer 2026 (Mountain View, CA) in Mountain View, CA | LinkedIn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n      Skip to main content\n    \n\n\n\nLinkedIn\n\n\n\n\n\n\n\n              AI/ML Engineer Intern, Master's - Summer 2026 (Mountain View, CA) in Pune\n \nExpand search\n\n\n\n\n\n\nThis button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. \n \n\n\n\n                        Jobs\n                    \n\n\n\n                        People\n                    \n\n\n\n                        Learning\n                    \n\n\n\n\n\n\n\n\n\n\n\n\nClear text\n\n\n\n\n\n\nClear text\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\nClear text\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\nClear text\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nClear text\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Sign i

In [21]:
print(docs[0].page_content)







 
 









LinkedIn hiring AI/ML Engineer Intern, Master's - Summer 2026 (Mountain View, CA) in Mountain View, CA | LinkedIn






















 





      Skip to main content
    



LinkedIn







              AI/ML Engineer Intern, Master's - Summer 2026 (Mountain View, CA) in Pune
 
Expand search






This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. 
 



                        Jobs
                    



                        People
                    



                        Learning
                    












Clear text






Clear text
















 





Clear text









 





Clear text















Clear text















          Sign in
      

      Join now
 






 










AI/ML Engineer Intern, Master's - Summer 2026 (Mountain View, CA)

LinkedIn
Mountain View, CA



        Apply
      









AI/M

In [15]:
info_schema = {
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "JobDescription",
  "type": "object",
  "additionalProperties":  {
    "type": "boolean"
  },

  "required": [
    "job_title",
    "company_name",
    "raw_description"
  ],

  "properties": {
    "job_title": {
      "type": "string",
      "description": "Title of the job position"
    },

    "company_name": {
      "type": "string",
      "description": "Name of the hiring company"
    },

    "location": {
      "type": ["string", "null"],
      "description": "Job location (city, country, remote, hybrid, etc.)"
    },

    "employment_type": {
      "type": ["string", "null"],
      "description": "Type of employment such as Full-time, Part-time, Contract, Internship"
    },

    "experience_level": {
      "type": ["string", "null"],
      "description": "Experience level such as Entry, Mid, Senior"
    },

    "posted_date": {
      "type": ["string", "null"],
      "description": "Date when the job was posted (raw text or ISO format if available)"
    },

    "raw_description": {
      "type": "string",
      "description": "Full unprocessed job description text scraped from the job page"
    },

    "responsibilities": {
      "type": "array",
      "description": "List of key responsibilities extracted from the description",
      "items": {
        "type": "string"
      },
      "default": []
    },

    "required_skills": {
      "type": "array",
      "description": "List of mandatory technical or professional skills required",
      "items": {
        "type": "string"
      },
      "default": []
    },

    "preferred_skills": {
      "type": "array",
      "description": "List of optional or nice-to-have skills",
      "items": {
        "type": "string"
      },
      "default": []
    },

    "experience_required": {
      "type": ["string", "null"],
      "description": "Years or type of experience required (e.g., '3+ years in backend development')"
    },

    "education_required": {
      "type": ["string", "null"],
      "description": "Minimum education qualification required for the role"
    },

    "company_industry": {
      "type": ["string", "null"],
      "description": "Industry or domain of the company (e.g., FinTech, Healthcare)"
    },

    "benefits": {
      "type": "array",
      "description": "List of benefits offered with the job",
      "items": {
        "type": "string"
      },
      "default": []
    }
  }
}


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Bug_finder =llm.with_structured_output(info_schema)

/workspaces/resume-builder/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:2073: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [17]:
system_prompt = """You are a job description scraping agent.

Your task is to extract job-related information from a given job posting page or text
and output it strictly in the provided JSON schema.

CRITICAL RULES:
1. You MUST NOT rewrite, summarize, paraphrase, clean, or alter the raw job description.
2. The field "raw_description" must contain the full job description text exactly as provided,
   preserving all wording, formatting, and information.
3. Do NOT infer or hallucinate missing information.
4. If a field is not present in the source, set it to null or an empty list as defined by the schema.
5. Output ONLY valid JSON that strictly conforms to the schema.
6. Do NOT include explanations, comments, or extra text outside the JSON.
7. Do NOT add any fields not defined in the schema.

ROLE BEHAVIOR:
- Act strictly as a scraper, not an analyzer or editor.
- Extract metadata (job title, company, location, etc.) only if explicitly present.
- Preserve factual accuracy over completeness.

Failure to follow these rules is considered an incorrect output.
"""

In [26]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
Bug_finder.invoke([SystemMessage(content = system_prompt ),HumanMessage(content = docs[0].page_content)])

{'job_title': "AI/ML Engineer Intern, Master's - Summer 2026",
 'company_name': 'LinkedIn',
 'location': 'Mountain View, CA',
 'employment_type': 'Internship',
 'experience_level': None,
 'posted_date': '1 week ago',
 'raw_description': 'Company DescriptionLinkedIn is the world’s largest professional network, built to create economic opportunity for every member of the global workforce. Our products help people make powerful connections, discover exciting opportunities, build necessary skills, and gain valuable insights every day. We’re also committed to providing transformational opportunities for our own employees by investing in their growth. We aspire to create a culture that’s built on trust, care, inclusion, and fun – where everyone can succeed.Join us to transform the way the world works.Job DescriptionOur internship roles will be based in Mountain View, CA.At LinkedIn, our approach to flexible work is centered on trust and optimized for culture, connection, clarity, and the evo